In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
# autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
autolib_dir = '/'.join(nb_dir.split('/')[:-1])
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

/home/kai/data/shiyi/AlphaBoosting/automl/automl_app
['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [2]:
from automl_libs import base_layer_utils, stacknet
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance'] 

In [4]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
#                                        fe.cumulative_count, 
#                                        fe.reverse_cumulative_count, 
#                                        fe.variance, fe.count_std_over_mean], 
#                                       2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
features_to_gen = get_features_to_gen([fe.count], 2, 2)#len(categorical_features))

features_to_gen 

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DepTime'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'UniqueCarrier'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Origin'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Dest'],
  'function': <function automl_libs.feature_engineering.co

In [5]:
len(features_to_gen)

28

In [6]:
project_path = './' 

In [7]:
from automl_app import logger_config
import logging
logger_config.config(project_path+'project.log', file_loglevel=logging.INFO)

In [8]:
def params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'svc':
        params = {
            'C': 0.911#np.random.rand(),
#             'metric': 'roc_auc'
        }
    elif model == 'logreg':
        params = {
            'penalty': np.random.choice(['l2']),
            'dual': np.random.choice([True, False]),
            'C': 1,#np.random.rand(),
            'metric': 'roc_auc'
        }
    elif model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 10, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc'
#             'objective': 'binary',
#             'boosting': 'gbdt',
#             'num_boost_round': 10, # ignored in params. extract it and put it in input arguments in train or cv explicitly
#                                    # seems to work fine as the upper limit when combined with early_stopping_round
#             'learning_rate': 0.01,
#             'num_leaves': 50,
#             'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
#             'max_depth': 5, # no limit. This is used to deal with over-fitting when #data is small.
#             'min_split_gain': 0.02,
#             'min_child_weight': 18.6,
#             'feature_fraction': 0.1,
#             'feature_fraction_seed': 1992,
#             'early_stopping_round':10,
#             'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
#             'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
#             'bagging_seed': 1992,
#             'scale_pos_weight': 1,
#             'lambda_l1': 0.8,
#             'lambda_l2': 0.6,
#             'metric': 'auc'
        }
    elif model == 'xgb':
        params = {
            'eta': 0.1,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'booster': 'gbtree',
            'tree_method': 'hist',
            'max_bin': 8,
            'scale_pos_weight': 1,
            'seed': seed,
            'nthread': 10,
            'max_depth': 6,
            'min_child_weight': 22,
            'gamma': 0,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'colsample_bylevel': 0.675,
            'alpha': 0,
            'lambda': 0,
            'num_boost_round': 50,
            'early_stopping_rounds': 1
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': np.random.randint(2,10),
            'lr_init': 0.01,
            'lr_fin': np.random.randint(1,5)/1000,  # if == lr_init, then no lr decay
            'batch_size': np.random.choice([128, 256, 512, 1024]),
            "pred_batch_size": 5000,
            'max_ep': 100,
            'patience': 10, #np.random.randint(10, 25),
            'cat_emb_outdim': 30, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
                                  # then assige embed_outdim_dict to cat_emb_outdim
            'cat_emb_drop_rate': np.random.randint(1,4)/10, 
            'num_layers_dense_units': [1000, 500, 100],
            'num_layers_drop_rate': np.random.randint(2,6)/10,
            'combined_layers_dense_units': [100, 50],
            'combined_layers_drop_rate': np.random.randint(1,3)/10,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    elif model == 'stacknet_layer2_nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.01, # if == lr_init, then no lr decay
            'batch_size': 128,
            "pred_batch_size": 50000,
            'best_epoch': 1,
            'patience': 1,
            'categorical_feature': [],
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'num_layers_dense_units': [],
            'combined_layers_dense_units': [10, 5],
            'combined_layers_drop_rate': 0,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    return params, seed

In [9]:
p, s = params_gen('stacknet_layer2_nn')
p, s

({'batch_size': 128,
  'best_epoch': 1,
  'cat_emb_outdim': 50,
  'categorical_feature': [],
  'combined_layers_dense_units': [10, 5],
  'combined_layers_drop_rate': 0,
  'ep_for_lr': 1,
  'int_list': ['num_layers_dense_units', 'combined_layers_dense_units'],
  'lr_fin': 0.01,
  'lr_init': 0.01,
  'mode': 'max',
  'monitor': 'val_auc',
  'nn_seed': 25446705,
  'num_layers_dense_units': [],
  'patience': 1,
  'pred_batch_size': 50000},
 25446694)

In [10]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, params_gen)

2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #22  | ==========START==========
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #67  | Run record file [./output/last_run_record.json] found. Continue from the previous run...
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #96  | generate todo list
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #112 | STAGE: FEATURE_ENGINEERING
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #116 | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #120 | STAGE: CONCAT_DATA
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting.__init__             | #124 | STAGE: GRID_SEARCH
2018-07-16 21:33:22 | INFO    | automl_app.app.AlphaBoosting._get_final_data      | #336 | DEBUG mode is on, 5.0% of train,val da

[0]	train-auc:0.700651+0.00094298	test-auc:0.692028+0.00526853
[1]	train-auc:0.707765+0.00210751	test-auc:0.695739+0.00206861
[2]	train-auc:0.713592+0.000709087	test-auc:0.700117+0.00366929
[3]	train-auc:0.715222+0.000690624	test-auc:0.701831+0.00292729
[4]	train-auc:0.717169+0.00115498	test-auc:0.702399+0.00357703
[5]	train-auc:0.718225+0.00176	test-auc:0.703125+0.00352992
[6]	train-auc:0.719494+0.00183939	test-auc:0.704592+0.00305248
[7]	train-auc:0.721796+0.00107919	test-auc:0.705569+0.00280796
[8]	train-auc:0.722624+0.00147927	test-auc:0.706131+0.00268604
[9]	train-auc:0.72388+0.00136941	test-auc:0.706991+0.00300239
[10]	train-auc:0.725277+0.00166173	test-auc:0.707495+0.00289461
[11]	train-auc:0.726271+0.00153619	test-auc:0.708318+0.00273878
[12]	train-auc:0.727591+0.00168341	test-auc:0.708787+0.00292673
[13]	train-auc:0.728316+0.00136884	test-auc:0.708944+0.00296979
[14]	train-auc:0.729538+0.00126361	test-auc:0.709387+0.00315185
[15]	train-auc:0.731026+0.00125891	test-auc:0.710009

2018-07-16 21:33:25 | INFO    | automl_libs.grid_search._xgb_gs              | #320 | val_auc: 0.72358 | train_auc: 0.76360 (cv)
2018-07-16 21:33:25 | INFO    | automl_libs.grid_search._xgb_gs              | #342 | [do_preds] is True, generating predictions ...
2018-07-16 21:33:25 | INFO    | automl_libs.grid_search._xgb_gs              | #344 | Retrain model using best_round [50] and all data...


[0]	all_data-auc:0.647336
[10]	all_data-auc:0.721826
[20]	all_data-auc:0.731441
[30]	all_data-auc:0.740421
[40]	all_data-auc:0.75008
[49]	all_data-auc:0.758169


2018-07-16 21:33:26 | INFO    | automl_libs.grid_search._xgb_gs              | #352 | Training done. Iteration: 50 | train_auc: 0.75817
2018-07-16 21:33:26 | INFO    | automl_libs.grid_search._xgb_gs              | #357 | (_nn_gs) roc of test: 0.7156622156031474
2018-07-16 21:33:26 | INFO    | automl_libs.grid_search._xgb_gs              | #362 | XGB predictions(bNsu) saved in ./output/gs_saved_preds/.
2018-07-16 21:33:26 | INFO    | automl_libs.grid_search.gs                   | #70  | ./output/xgb_ordinal2_grid_search.csv updated
2018-07-16 21:33:26 | INFO    | automl_libs.grid_search.gs                   | #35  | Grid search xgb. round 2 of 2


...

2018-07-16 21:33:31 | INFO    | automl_libs.base_layer_utils.get_oof              | #448 | metric of fold 4: 0.727201355315896
2018-07-16 21:33:31 | INFO    | automl_libs.base_layer_utils.get_oof              | #434 | processing fold 5 of 5...
2018-07-16 21:33:31 | INFO    | automl_libs.stack_layer_estimator.XgboostBLE.train                | #208 | No evaluation set, thus not possible to use early stopping. Please train with your best params.


[0]	train-auc:0.70274
[10]	train-auc:0.726617
[20]	train-auc:0.735194
[30]	train-auc:0.746205
[40]	train-auc:0.756
[49]	train-auc:0.762957


2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.get_oof              | #448 | metric of fold 5: 0.7184368953492759
2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.compute_layer1_oof   | #527 | oof is done
2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.compute_layer1_oof   | #531 | Training using all data and gen prediction for submission...
2018-07-16 21:33:32 | INFO    | automl_libs.stack_layer_estimator.XgboostBLE.train                | #208 | No evaluation set, thus not possible to use early stopping. Please train with your best params.


[0]	train-auc:0.696268
[10]	train-auc:0.723615
[20]	train-auc:0.7335
[30]	train-auc:0.743332
[40]	train-auc:0.752675
[49]	train-auc:0.760036


2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.update_report        | #280 | StackNet report updated: syXQ__XGB_ordinal2_layer1: oof_cv_score => 0.7214209972736969
2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.add_score            | #299 | syXQ__XGB_ordinal2_layer1 found in repo. Update score from 0 to 0.72383
2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.update_report        | #280 | StackNet report updated: syXQ__XGB_ordinal2_layer1: gs_val_auc => 0.7238258000000001
2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.update_report        | #280 | StackNet report updated: syXQ__XGB_ordinal2_layer1: test_score => 0.7157585213683268
2018-07-16 21:33:32 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.save                 | #392 | StackNet data saved for: ['Ffk4__LGB_ordinal_layer1', '8jnJ__LGB_ordinal_layer1', 'GBVc__XGB_ordinal_layer1', '